In [1]:
import pandas as pd
import re
import os
import numpy as np
import datetime

In [2]:
csv_list = []

LUIGI_LOG_PATH = '/usr/local/trustyou/home/andream/nfs/processmining/pm4pyexample/'
LUIGI_LOG_PATH = '/usr/local/trustyou/home/andream/nfs/processmining/minilogs/daily/logs/'
#LUIGI_LOG_PATH = '/usr/local/trustyou/home/andream/nfs/processmining/minilogs/table-precomp/logs/tch/'
#LUIGI_LOG_PATH='/usr/local/trustyou/home/andream/nfs/processmining/minilogs/table-precomp/logs/october/mongo/'
for filename in os.listdir(LUIGI_LOG_PATH):
    if filename.endswith('.csv'):
        log_path = LUIGI_LOG_PATH+'/'+filename
        csv_list.append(filename)
appended_df = pd.DataFrame()
#appended_df
csv_list.sort()
csv_list

['daily.2019-09-01_09-29-01.csv',
 'daily.2019-09-03_03-44-01.csv',
 'daily.2019-09-04_23-23-01.csv',
 'daily.2019-09-06_20-34-01.csv',
 'daily.2019-09-09_01-34-02.csv',
 'daily.2019-09-11_05-40-02.csv',
 'daily.2019-09-13_04-11-01.csv',
 'daily.2019-09-15_01-41-01.csv',
 'daily.2019-09-16_19-44-01.csv',
 'daily.2019-09-18_14-08-01.csv',
 'daily.2019-09-20_12-49-01.csv',
 'daily.2019-09-22_07-40-01.csv',
 'daily.2019-09-24_05-05-02.csv']

In [3]:
from difflib import SequenceMatcher 
  
def longestSubstring(str1,str2): 
  
     # initialize SequenceMatcher object with  
     # input string 
     seqMatch = SequenceMatcher(None,str1,str2) 
  
     # find match of longest sub-string 
     # output will be like Match(a=0, b=0, size=5) 
     match = seqMatch.find_longest_match(0, len(str1), 0, len(str2)) 
  
     # print longest substring 
     if (match.size!=0): 
          result = str1[match.a: match.a + match.size]
          print (result)  
          return result
     else: 
          print ('No longest common sub-string found')

In [4]:
PROCESS_NAME = longestSubstring(csv_list[0],csv_list[-1])
OUTPUT_CATT_PATH = LUIGI_LOG_PATH+'/../catts_'+str(PROCESS_NAME)+'.csv'
print(OUTPUT_CATT_PATH)

daily.2019-09-
/usr/local/trustyou/home/andream/nfs/processmining/minilogs/daily/logs//../catts_daily.2019-09-.csv


In [5]:
for filename in os.listdir(LUIGI_LOG_PATH):
    if filename.endswith('.csv'):
        log_path = LUIGI_LOG_PATH+'/'+filename
        print('Preprocessing... ', filename)
        df = pd.read_csv(log_path, index_col=0)
        df = df[['timestamp','task','state']]
        df['source_file'] = filename

        df['state'][0]='start'
        df['task'][0]='AllTasks'

        df['state'][len(df)-1]='done'
        df['task'][len(df)-1]='AllTasks'
        
        appended_df = appended_df.append(df)
        continue
    else:
        continue

print(len(appended_df))
appended_df.head()

Preprocessing...  daily.2019-09-04_23-23-01.csv
Preprocessing...  daily.2019-09-09_01-34-02.csv
Preprocessing...  daily.2019-09-16_19-44-01.csv
Preprocessing...  daily.2019-09-20_12-49-01.csv
Preprocessing...  daily.2019-09-24_05-05-02.csv
Preprocessing...  daily.2019-09-11_05-40-02.csv
Preprocessing...  daily.2019-09-13_04-11-01.csv
Preprocessing...  daily.2019-09-06_20-34-01.csv
Preprocessing...  daily.2019-09-18_14-08-01.csv
Preprocessing...  daily.2019-09-15_01-41-01.csv
Preprocessing...  daily.2019-09-03_03-44-01.csv
Preprocessing...  daily.2019-09-22_07-40-01.csv
Preprocessing...  daily.2019-09-01_09-29-01.csv
31469


,timestamp,task,state,source_file
0,2019-09-04 23:23:20,AllTasks,start,daily.2019-09-04_23-23-01.csv
1,2019-09-04 23:35:08,"DumpTask(date=2019-09-04_23-23-01, prev_date=2...",start,daily.2019-09-04_23-23-01.csv
2,2019-09-04 23:35:08,"DumpTask(date=2019-09-04_23-23-01, prev_date=2...",start,daily.2019-09-04_23-23-01.csv
3,2019-09-04 23:35:08,"DumpTask(date=2019-09-04_23-23-01, prev_date=2...",start,daily.2019-09-04_23-23-01.csv
4,2019-09-04 23:35:08,"ProxySetupTask(date=2019-09-04_23-23-01, prev_...",start,daily.2019-09-04_23-23-01.csv


In [6]:
def get_task(task_call):
    if isinstance(task_call, float):
        return task_call
    return task_call.split('(')[0]

In [7]:
df = appended_df
df['task_call'] = df['task']
df['task_name'] = np.where(df['task'].notnull(), df['task'].apply(get_task), df['task'])
df = df[['task_name','state','timestamp','task_call', 'source_file']]

df['start_time'] = np.where(df['state']=='start', df['timestamp'], np.nan)
df['end_time'] = np.where(df['state']=='done', df['timestamp'], np.nan)

start_df = df[df['state']=='start']
print('Start entries: ',len(start_df))

end_df = df[df['state']=='done']
print('End entries: ',len(end_df))

df.head()

Start entries:  15729
End entries:  15699


,task_name,state,timestamp,task_call,source_file,start_time,end_time
0,AllTasks,start,2019-09-04 23:23:20,AllTasks,daily.2019-09-04_23-23-01.csv,2019-09-04 23:23:20,NaN
1,DumpTask,start,2019-09-04 23:35:08,"DumpTask(date=2019-09-04_23-23-01, prev_date=2...",daily.2019-09-04_23-23-01.csv,2019-09-04 23:35:08,NaN
2,DumpTask,start,2019-09-04 23:35:08,"DumpTask(date=2019-09-04_23-23-01, prev_date=2...",daily.2019-09-04_23-23-01.csv,2019-09-04 23:35:08,NaN
3,DumpTask,start,2019-09-04 23:35:08,"DumpTask(date=2019-09-04_23-23-01, prev_date=2...",daily.2019-09-04_23-23-01.csv,2019-09-04 23:35:08,NaN
4,ProxySetupTask,start,2019-09-04 23:35:08,"ProxySetupTask(date=2019-09-04_23-23-01, prev_...",daily.2019-09-04_23-23-01.csv,2019-09-04 23:35:08,NaN


In [8]:
tt = pd.merge(start_df, end_df, on=['task_call', 'source_file'], how='outer')
tt = tt[tt['start_time_x'].notnull()]
tt = tt[tt['end_time_y'].notnull()]

tt['case'] = tt['source_file']
tt['activity'] = tt['task_name_x']
tt['start_time'] = tt['start_time_x']
tt['end_time'] = tt['end_time_y']
print(len(tt))

15711


## Save catt to csv

In [9]:
catt = tt[['case', 'activity','start_time','end_time']]

print(len(catt))
print(catt.head())
catt.to_csv(OUTPUT_CATT_PATH, index = None, header=True)


15711
                            case        activity           start_time  \
0  daily.2019-09-04_23-23-01.csv        AllTasks  2019-09-04 23:23:20   
1  daily.2019-09-04_23-23-01.csv        DumpTask  2019-09-04 23:35:08   
2  daily.2019-09-04_23-23-01.csv        DumpTask  2019-09-04 23:35:08   
3  daily.2019-09-04_23-23-01.csv        DumpTask  2019-09-04 23:35:08   
4  daily.2019-09-04_23-23-01.csv  ProxySetupTask  2019-09-04 23:35:08   

              end_time  
0  2019-09-06 20:33:05  
1  2019-09-04 23:36:33  
2  2019-09-05 00:00:34  
3  2019-09-04 23:35:58  
4  2019-09-04 23:51:32  


In [10]:
pd.options.display.max_colwidth = 100
w_call = tt[['case', 'activity','start_time','end_time', 'task_call']]
grouped = w_call.groupby(['task_call'],as_index=False)
print(len(grouped))
#grouped.head()
#grouped = grouped[grouped['activity']!='CrawlTask']
#grouped.size()

15687


In [11]:
def get_duration(start_time, end_time):
    start = datetime.datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')
    end = datetime.datetime.strptime(end_time, '%Y-%m-%d %H:%M:%S')
    duration = abs(end - start)
    return duration
#get_duration(ex['timestamp'][10],ex['timestamp'][1])

In [12]:
w_duration = tt.copy()
w_duration = w_duration[['case', 'activity','start_time','end_time']]
w_duration['duration'] = w_duration.apply(lambda row: str(get_duration(row['start_time'],row['end_time'])), axis=1)
w_duration['t_duration']= w_duration.apply(lambda row: (get_duration(row['start_time'],row['end_time']).total_seconds())/60/60, axis=1)


w_duration = w_duration[['case','activity','start_time','end_time', 'duration', 't_duration']]
print(len(w_duration))
w_duration.head()

15711


,case,activity,start_time,end_time,duration,t_duration
0,daily.2019-09-04_23-23-01.csv,AllTasks,2019-09-04 23:23:20,2019-09-06 20:33:05,"1 day, 21:09:45",45.162500
1,daily.2019-09-04_23-23-01.csv,DumpTask,2019-09-04 23:35:08,2019-09-04 23:36:33,0:01:25,0.023611
2,daily.2019-09-04_23-23-01.csv,DumpTask,2019-09-04 23:35:08,2019-09-05 00:00:34,0:25:26,0.423889
3,daily.2019-09-04_23-23-01.csv,DumpTask,2019-09-04 23:35:08,2019-09-04 23:35:58,0:00:50,0.013889
4,daily.2019-09-04_23-23-01.csv,ProxySetupTask,2019-09-04 23:35:08,2019-09-04 23:51:32,0:16:24,0.273333


In [13]:
top_long_tasks = w_duration.head(5000)


In [14]:
max_duration = top_long_tasks.groupby(['activity'], sort=False)['t_duration'].max().reset_index()
#.reset_index(name='max_duration').sort_values(by=['max_duration'], ascending=False)
counts = top_long_tasks.groupby(['activity']).size().reset_index(name='counts').sort_values(by=['counts'], ascending=False)
act_group = pd.merge(counts, max_duration, on=['activity'], how='outer')
act_group = act_group.sort_values(by=['counts'], ascending = False)
print(len(act_group))
act_group

67


,activity,counts,t_duration
0,ExtractTask,2014,1.964722
1,CrawlTask,1992,6.859167
2,DumpTask,189,3.421111
3,ConvertDumpTask,166,0.354444
4,GroupTask,31,22.866944
5,QCValidateTask,24,0.098333
6,ExtractChunkTask,24,0.052500
7,CrawlChunkTask,24,0.045833
32,UserDemographicsFullUpdateTask,12,0.000833
26,StripPIITask,12,0.025833


In [15]:
csv_list = []
for filename in os.listdir(LUIGI_LOG_PATH+'../'):
    if filename.endswith('.csv'):
        print(filename)

catts_daily.2019-09-.csv
catts_daily.2019-09-01_09-29-01.csv


### Sanity Check

In [16]:
csv_list = []
appended_df = pd.DataFrame()
LUIGI_LOG_PATH='/usr/local/trustyou/home/andream/nfs/processmining/minilogs/table-precomp/logs/october/'
#LUIGI_LOG_PATH = '/usr/local/trustyou/home/andream/nfs/processmining/minilogs/daily/logs/'
#LUIGI_LOG_PATH = '/usr/local/trustyou/home/andream/nfs/processmining/minilogs/table-precomp/logs/tch/'
for filename in os.listdir(LUIGI_LOG_PATH):
    if filename.endswith('.csv') and filename.startswith('catts'):
        log_path = LUIGI_LOG_PATH+'/'+filename
        csv_list.append(filename)
        df = pd.read_csv(log_path, index_col=False)
        appended_df = appended_df.append(df)

#appended_df
csv_list.sort()
print(csv_list)
print(len(appended_df))
appended_df.head()

['catts_group.hotel.2019-10-.csv']
7128


,case,activity,start_time,end_time
0,group.hotel.2019-10-15_04-18-01_01.mongo.csv,AllTasks,2019-10-16 02:07:56,2019-10-16 23:21:42
1,group.hotel.2019-10-15_04-18-01_01.mongo.csv,CategorySentimentTask,2019-10-16 02:08:18,2019-10-16 02:08:19
2,group.hotel.2019-10-15_04-18-01_01.mongo.csv,MatchIncDumpTask,2019-10-16 02:08:18,2019-10-16 02:08:48
3,group.hotel.2019-10-15_04-18-01_01.mongo.csv,MgmtResponseIncDumpTask,2019-10-16 02:08:18,2019-10-16 02:09:51
4,group.hotel.2019-10-15_04-18-01_01.mongo.csv,ReviewIncDumpTask,2019-10-16 02:08:18,2019-10-16 02:08:42


## Cleaning paramters

In [17]:
task_calls = tt.copy()[['case', 'activity', 'task_call', 'start_time', 'end_time']]
print(len(task_calls))
print(task_calls.head())

15711
                            case        activity  \
0  daily.2019-09-04_23-23-01.csv        AllTasks   
1  daily.2019-09-04_23-23-01.csv        DumpTask   
2  daily.2019-09-04_23-23-01.csv        DumpTask   
3  daily.2019-09-04_23-23-01.csv        DumpTask   
4  daily.2019-09-04_23-23-01.csv  ProxySetupTask   

                                                                                             task_call  \
0                                                                                             AllTasks   
1  DumpTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=prep, sql_filename=daily...   
2  DumpTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=prep, sql_filename=regul...   
3  DumpTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=prep, sql_filename=repro...   
4                ProxySetupTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=prep)\n   

            start_time             end_time 

In [18]:
counts = task_calls.groupby(['activity']).size().reset_index(name='counts').sort_values(by=['counts'], ascending=False)
counts = counts.sort_values(by=['counts'], ascending = False)
print(len(task_calls),' lines in catt')
print(counts.head(), '\n')

#appended = appended.head(100)
#len(appended)
unique_act = task_calls['activity'].unique().tolist()
print(len(unique_act), ' different activities')

unique_trace = task_calls['case'].unique().tolist()
print(len(unique_trace), ' cases\n')

unique_task_call = task_calls['task_call'].unique().tolist()
print(len(unique_task_call), 'unique task_calls')
print(len(task_calls)-len(unique_task_call), 'duplicated task_calls')

pd.set_option('display.max_colwidth', -1)
dfObj = pd.DataFrame(task_calls, columns=['case','task_call','activity','start_time', 'end_time'])
duplicated_task_calls = dfObj[dfObj.duplicated(['case','task_call','end_time'])].sort_values(by=['task_call'])['task_call'].unique().tolist()
print(len(duplicated_task_calls),' unique duplicated task_calls')

15711  lines in catt
           activity  counts
19      ExtractTask    6346
14        CrawlTask    6326
17         DumpTask     585
8   ConvertDumpTask     533
24        GroupTask      89 

67  different activities
13  cases

15687 unique task_calls
24 duplicated task_calls
9  unique duplicated task_calls


In [289]:
def extract_parameters(task_call):
    task_split = task_call.split('(', 1)
    parameters = {}
    next_key = None
    key = None
    if len(task_split)>1: 
        reversed_parameters = [''.join(reversed(element)) for element in task_split[1:]][0].split(')', 1)[1:]
        forward_parameters = [''.join(reversed(element)) for element in reversed_parameters][0]
        parameters_call = forward_parameters.split('=')
        #print(task_call)
        for i, element in enumerate(parameters_call): 
            if next_key is None and key is None: 
                key = element
                continue
            if next_key is not None:
                key = next_key
            if element.startswith('('):
                next_key = re.search('\), (.*)', element)
                value = re.search('(\(.*)\), ', element)
            else: 
                next_key = re.search(', (.*)', element)
                value = re.search('(.*), ', element)
            if next_key is None: 
                #Find out if elem is key or value
                value = element
            else:
                next_key = next_key.group(1)
                if element.startswith('('):
                    value = value.group(0)[:-2]
                else: 
                    value = value.group(1)
            #print('A: ', key, value)               
            parameters[key] = value
    return parameters

#print(extract_parameters('AllTasks'))
#extract_parameters("DumpTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=prep, sql_filename=review, kvs=(('table_name', u'review_2018_3'),), target_filename=review_2018_3, db_host=db.trustyou.com, db_port=5432, db_user=daily, db_name=ty_analytic)")

In [290]:
#task_calls[task_calls['task_call']=='CrawlTask(date=2019-09-15_01-41-01, prev_date=2019-09-13_04-11-01, chunk=02, partition=00007, crawler=creepy-crawly)\n']
#task_calls[task_calls['task_call']=='GroupTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=02, datacenter=eu, inc=True, deploy=False, datastore=mongo)\n']
#task_calls[task_calls['task_call']=='AllTasks'].sort_values(by='case')
#print(task_calls.head())
task_calls['parameters'] = task_calls.apply(lambda row: extract_parameters(row['task_call']), axis=1)
#print(task_calls[['task_call', 'parameters']].head(50))
print(task_calls['parameters'][1]['date'])
print(task_calls.iloc[1]['parameters']['date'])
#print(task_calls['parameters'][1]['date'])

2019-09-04_23-23-01
2019-09-04_23-23-01


In [292]:
pd.set_option('display.max_colwidth', -1)
#task_calls[task_calls['activity']=='CrawlChunkTask'][['case','activity','parameters']].head()
#print(task_calls.iloc[1]['parameters'])

In [293]:
pd.set_option('display.max_columns', None)

expanded_params = pd.concat([task_calls[:], task_calls['parameters'].apply(pd.Series)], axis=1)
len(expanded_params)
#expanded_params.head()

15711

In [294]:
expanded_params = expanded_params.replace('None', np.nan)
print(expanded_params.columns)
#expanded_params.head()

Index(['case', 'activity', 'task_call', 'start_time', 'end_time', 'parameters',
       'date', 'prev_date', 'chunk', 'sql_filename', 'kvs', 'target_filename',
       'db_host', 'db_port', 'db_user', 'db_name', 'output_format',
       'filename_chunk', 'what', 'crawler', 'partition', 'sql_templ_filename',
       'parent', 'schema', 'archive_hdfs_filepath', 'host', 'port',
       'namespace', 'table', 'datacenter', 'inc', 'deploy', 'datastore',
       'send_to'],
      dtype='object')


In [315]:
activity_selection = expanded_params[expanded_params['activity']=='CrawlTask'].sort_values(by='case')
activity_selection.columns

Index(['case', 'activity', 'task_call', 'start_time', 'end_time', 'parameters',
       'date', 'prev_date', 'chunk', 'sql_filename', 'kvs', 'target_filename',
       'db_host', 'db_port', 'db_user', 'db_name', 'output_format',
       'filename_chunk', 'what', 'crawler', 'partition', 'sql_templ_filename',
       'parent', 'schema', 'archive_hdfs_filepath', 'host', 'port',
       'namespace', 'table', 'datacenter', 'inc', 'deploy', 'datastore',
       'send_to'],
      dtype='object')

In [316]:
activity_selection = activity_selection.dropna(axis=1,how='all')

print(activity_selection.columns)
activity_selection.head()

Index(['case', 'activity', 'task_call', 'start_time', 'end_time', 'parameters',
       'date', 'prev_date', 'chunk', 'crawler', 'partition'],
      dtype='object')


,case,activity,task_call,start_time,end_time,parameters,date,prev_date,chunk,crawler,partition
15634,daily.2019-09-01_09-29-01.csv,CrawlTask,"CrawlTask(date=2019-09-01_09-29-01, prev_date=2019-08-30_13-45-01, chunk=02, partition=00000, crawler=creepy-crawly)\n",2019-09-02 16:01:35,2019-09-02 17:16:13,"{'date': '2019-09-01_09-29-01', 'prev_date': '2019-08-30_13-45-01', 'chunk': '02', 'partition': '00000', 'crawler': 'creepy-crawly'}",2019-09-01_09-29-01,2019-08-30_13-45-01,02,creepy-crawly,00000
14839,daily.2019-09-01_09-29-01.csv,CrawlTask,"CrawlTask(date=2019-09-01_09-29-01, prev_date=2019-08-30_13-45-01, chunk=01, partition=00058, crawler=creepy-crawly)\n",2019-09-01 15:01:53,2019-09-01 18:54:22,"{'date': '2019-09-01_09-29-01', 'prev_date': '2019-08-30_13-45-01', 'chunk': '01', 'partition': '00058', 'crawler': 'creepy-crawly'}",2019-09-01_09-29-01,2019-08-30_13-45-01,01,creepy-crawly,00058
14841,daily.2019-09-01_09-29-01.csv,CrawlTask,"CrawlTask(date=2019-09-01_09-29-01, prev_date=2019-08-30_13-45-01, chunk=01, partition=00031, crawler=creepy-crawly)\n",2019-09-01 15:02:28,2019-09-01 16:56:20,"{'date': '2019-09-01_09-29-01', 'prev_date': '2019-08-30_13-45-01', 'chunk': '01', 'partition': '00031', 'crawler': 'creepy-crawly'}",2019-09-01_09-29-01,2019-08-30_13-45-01,01,creepy-crawly,00031
14843,daily.2019-09-01_09-29-01.csv,CrawlTask,"CrawlTask(date=2019-09-01_09-29-01, prev_date=2019-08-30_13-45-01, chunk=01, partition=00000, crawler=creepy-crawly)\n",2019-09-01 15:05:36,2019-09-01 17:38:07,"{'date': '2019-09-01_09-29-01', 'prev_date': '2019-08-30_13-45-01', 'chunk': '01', 'partition': '00000', 'crawler': 'creepy-crawly'}",2019-09-01_09-29-01,2019-08-30_13-45-01,01,creepy-crawly,00000
14845,daily.2019-09-01_09-29-01.csv,CrawlTask,"CrawlTask(date=2019-09-01_09-29-01, prev_date=2019-08-30_13-45-01, chunk=01, partition=00085, crawler=creepy-crawly)\n",2019-09-01 15:10:47,2019-09-01 17:48:42,"{'date': '2019-09-01_09-29-01', 'prev_date': '2019-08-30_13-45-01', 'chunk': '01', 'partition': '00085', 'crawler': 'creepy-crawly'}",2019-09-01_09-29-01,2019-08-30_13-45-01,01,creepy-crawly,00085


In [317]:

non_str_columns = ['parameters']
column_selection = set(activity_selection.columns) - set(non_str_columns)

#counts = activity_selection.groupby(['db_port']).size().reset_index(name='counts').sort_values(by=['counts'])

print(len(activity_selection),'rows:\n')
print(activity_selection.head(1)[['case','activity','start_time','end_time']])

groups_description = pd.DataFrame(columns=['column','different_groups_per_column'])
for column in column_selection: 
    #print(column,len(clean_activity_selection.groupby([column]).size().reset_index(name='counts').sort_values(by=['counts'])))
    groups_description = groups_description.append([{'column': column, 'different_groups_per_column': len(activity_selection.groupby([column]).size().reset_index(name='counts').sort_values(by=['counts']))}])
groups_description = groups_description.reset_index()[['column','different_groups_per_column']].sort_values(by=['different_groups_per_column'])

groups_description

6326 rows:

                                case   activity           start_time  \
15634  daily.2019-09-01_09-29-01.csv  CrawlTask  2019-09-02 16:01:35   

                  end_time  
15634  2019-09-02 17:16:13  


,column,different_groups_per_column
2,activity,1
4,crawler,2
8,chunk,3
0,prev_date,13
3,case,13
6,date,13
1,partition,240
5,start_time,5159
7,end_time,6281
9,task_call,6325


In [318]:
similar_group_descr = groups_description.groupby(by=['different_groups_per_column']).size().reset_index(name='similar_group')
similar_group_counts = similar_group_descr[similar_group_descr['similar_group']>1]['different_groups_per_column'].tolist()
print('Similar number of groups with multiple appereances in Groupbys: ', similar_group_counts)

drop_candidates = []
for group in similar_group_counts: 
    parameters = groups_description[groups_description['different_groups_per_column']==group]['column'].tolist()
    drop_candidates.append(parameters)
print('Groups with similar number of groups: ', drop_candidates)


Similar number of groups with multiple appereances in Groupbys:  [13]
Groups with similar number of groups:  [['prev_date', 'case', 'date']]


In [319]:
def get_activity_new_name(old_name, column_key, column_value):
    activity_name = str(old_name)+'('+str(column_key)
    if column_value is np.nan: 
        activity_name = activity_name+'=None'
    else: 
        activity_name = activity_name+'='+str(column_value)
    activity_name = activity_name +')'
    return activity_name

def clean_insignificant_columns(df, drop_candidates):
    columns_to_drop = []
    verified_candidates = []
    number_of_cases = len(df.groupby(['case']))
    flag =0
    for group_candidates in drop_candidates:
        for candidate in group_candidates: 
            group_counts = df.groupby([candidate]).size().reset_index(name='counts').sort_values(by=['counts'])['counts'].tolist()
            if 'date' in candidate:
                verified_candidates.append(candidate)
            if len(group_counts)==1 or len(group_counts)==number_of_cases:
                if (all(item == group_counts[0] for item in group_counts) and (group_counts[0]*len(group_counts))== len(df)):
                    print(group_counts)
                    #or all(item == number_of_cases for item in group_counts)
                    verified_candidates.append(candidate)
            else:
                flag = 1
                break
        if flag:
            first_candidate_counts = df.groupby([group_candidates[0]]).size().reset_index(name='counts')['counts'].tolist()
            if (all(first_candidate_counts==df.groupby([item]).size().reset_index(name='counts')['counts'].tolist() for item in group_candidates)):
                all(verified_candidates.append(item) for item in group_candidates[:-1])
            flag = 0
    columns_to_drop = set(verified_candidates)-set(catt.columns)
    df = df.drop(columns_to_drop, axis=1)
    print('Dropped: ',columns_to_drop)
    
    for column in set(df.columns)-set(catt.columns)-set({'parameters', 'task_call'}):
        df['activity'] = df.apply(lambda row: get_activity_new_name(row['activity'], column, row[column]), axis=1)
    return df 

    
clean_dump = clean_insignificant_columns(activity_selection, drop_candidates)
counts = clean_dump.groupby(['activity']).size().reset_index(name='counts').sort_values(by=['counts'], ascending=False)
counts = counts.sort_values(by=['counts'], ascending = False)
counts

Dropped:  {'date', 'prev_date'}


,activity,counts
0,CrawlTask(partition=00000)(crawler=creepy-crawly)(chunk=01),13
442,CrawlTask(partition=00179)(crawler=creepy-crawly)(chunk=01),13
435,CrawlTask(partition=00172)(crawler=creepy-crawly)(chunk=01),13
436,CrawlTask(partition=00173)(crawler=creepy-crawly)(chunk=01),13
437,CrawlTask(partition=00174)(crawler=creepy-crawly)(chunk=01),13
438,CrawlTask(partition=00175)(crawler=creepy-crawly)(chunk=01),13
439,CrawlTask(partition=00176)(crawler=creepy-crawly)(chunk=01),13
440,CrawlTask(partition=00177)(crawler=creepy-crawly)(chunk=01),13
441,CrawlTask(partition=00178)(crawler=creepy-crawly)(chunk=01),13
443,CrawlTask(partition=00180)(crawler=creepy-crawly)(chunk=01),13


In [320]:
clean_dump.head()

,case,activity,task_call,start_time,end_time,parameters,chunk,crawler,partition
15634,daily.2019-09-01_09-29-01.csv,CrawlTask(partition=00000)(crawler=creepy-crawly)(chunk=02),"CrawlTask(date=2019-09-01_09-29-01, prev_date=2019-08-30_13-45-01, chunk=02, partition=00000, crawler=creepy-crawly)\n",2019-09-02 16:01:35,2019-09-02 17:16:13,"{'date': '2019-09-01_09-29-01', 'prev_date': '2019-08-30_13-45-01', 'chunk': '02', 'partition': '00000', 'crawler': 'creepy-crawly'}",02,creepy-crawly,00000
14839,daily.2019-09-01_09-29-01.csv,CrawlTask(partition=00058)(crawler=creepy-crawly)(chunk=01),"CrawlTask(date=2019-09-01_09-29-01, prev_date=2019-08-30_13-45-01, chunk=01, partition=00058, crawler=creepy-crawly)\n",2019-09-01 15:01:53,2019-09-01 18:54:22,"{'date': '2019-09-01_09-29-01', 'prev_date': '2019-08-30_13-45-01', 'chunk': '01', 'partition': '00058', 'crawler': 'creepy-crawly'}",01,creepy-crawly,00058
14841,daily.2019-09-01_09-29-01.csv,CrawlTask(partition=00031)(crawler=creepy-crawly)(chunk=01),"CrawlTask(date=2019-09-01_09-29-01, prev_date=2019-08-30_13-45-01, chunk=01, partition=00031, crawler=creepy-crawly)\n",2019-09-01 15:02:28,2019-09-01 16:56:20,"{'date': '2019-09-01_09-29-01', 'prev_date': '2019-08-30_13-45-01', 'chunk': '01', 'partition': '00031', 'crawler': 'creepy-crawly'}",01,creepy-crawly,00031
14843,daily.2019-09-01_09-29-01.csv,CrawlTask(partition=00000)(crawler=creepy-crawly)(chunk=01),"CrawlTask(date=2019-09-01_09-29-01, prev_date=2019-08-30_13-45-01, chunk=01, partition=00000, crawler=creepy-crawly)\n",2019-09-01 15:05:36,2019-09-01 17:38:07,"{'date': '2019-09-01_09-29-01', 'prev_date': '2019-08-30_13-45-01', 'chunk': '01', 'partition': '00000', 'crawler': 'creepy-crawly'}",01,creepy-crawly,00000
14845,daily.2019-09-01_09-29-01.csv,CrawlTask(partition=00085)(crawler=creepy-crawly)(chunk=01),"CrawlTask(date=2019-09-01_09-29-01, prev_date=2019-08-30_13-45-01, chunk=01, partition=00085, crawler=creepy-crawly)\n",2019-09-01 15:10:47,2019-09-01 17:48:42,"{'date': '2019-09-01_09-29-01', 'prev_date': '2019-08-30_13-45-01', 'chunk': '01', 'partition': '00085', 'crawler': 'creepy-crawly'}",01,creepy-crawly,00085


In [321]:
drop_candidates = groups_description[groups_description['different_groups_per_column']==13]['column'].tolist()

verified_candidates = []
for candidate in drop_candidates: 
    group_counts = clean_activity_selection.groupby([candidate]).size().reset_index(name='counts').sort_values(by=['counts'])['counts'].tolist()
    if all(item == group_counts[0] for item in group_counts) and (group_counts[0]*len(group_counts))== len(clean_activity_selection): 
        verified_candidates.append(candidate)  
columns_to_drop = set(verified_candidates)-set(catt.columns)
print(columns_to_drop)
clean_case_activity_selection = clean_activity_selection.drop(columns_to_drop, axis=1)

clean_case_activity_selection.head()

{'date', 'prev_date'}


,case,activity,task_call,start_time,end_time,parameters,chunk,sql_filename,kvs,target_filename
14771,daily.2019-09-01_09-29-01.csv,DumpTask,"DumpTask(date=2019-09-01_09-29-01, prev_date=2019-08-30_13-45-01, chunk=prep, sql_filename=source, kvs=None, target_filename=None, db_host=db.trustyou.com, db_port=5432, db_user=daily, db_name=ty_analytic)\n",2019-09-01 14:10:06,2019-09-01 14:11:53,"{'date': '2019-09-01_09-29-01', 'prev_date': '2019-08-30_13-45-01', 'chunk': 'prep', 'sql_filename': 'source', 'kvs': 'None', 'target_filename': 'None', 'db_host': 'db.trustyou.com', 'db_port': '5432', 'db_user': 'daily', 'db_name': 'ty_analytic'}",prep,source,NaN,NaN
14608,daily.2019-09-01_09-29-01.csv,DumpTask,"DumpTask(date=2019-09-01_09-29-01, prev_date=2019-08-30_13-45-01, chunk=prep, sql_filename=review, kvs=(('table_name', u'review_2017_1'),), target_filename=review_2017_1, db_host=db.trustyou.com, db_port=5432, db_user=daily, db_name=ty_analytic)\n",2019-09-01 12:03:41,2019-09-01 12:24:47,"{'date': '2019-09-01_09-29-01', 'prev_date': '2019-08-30_13-45-01', 'chunk': 'prep', 'sql_filename': 'review', 'kvs': '(('table_name', u'review_2017_1'),)', 'target_filename': 'review_2017_1', 'db_host': 'db.trustyou.com', 'db_port': '5432', 'db_user': 'daily', 'db_name': 'ty_analytic'}",prep,review,"(('table_name', u'review_2017_1'),)",review_2017_1
14606,daily.2019-09-01_09-29-01.csv,DumpTask,"DumpTask(date=2019-09-01_09-29-01, prev_date=2019-08-30_13-45-01, chunk=prep, sql_filename=review, kvs=(('table_name', u'review_2016_4'),), target_filename=review_2016_4, db_host=db.trustyou.com, db_port=5432, db_user=daily, db_name=ty_analytic)\n",2019-09-01 11:56:56,2019-09-01 12:18:18,"{'date': '2019-09-01_09-29-01', 'prev_date': '2019-08-30_13-45-01', 'chunk': 'prep', 'sql_filename': 'review', 'kvs': '(('table_name', u'review_2016_4'),)', 'target_filename': 'review_2016_4', 'db_host': 'db.trustyou.com', 'db_port': '5432', 'db_user': 'daily', 'db_name': 'ty_analytic'}",prep,review,"(('table_name', u'review_2016_4'),)",review_2016_4
14604,daily.2019-09-01_09-29-01.csv,DumpTask,"DumpTask(date=2019-09-01_09-29-01, prev_date=2019-08-30_13-45-01, chunk=prep, sql_filename=review, kvs=(('table_name', u'review_2yold'),), target_filename=review_2yold, db_host=db.trustyou.com, db_port=5432, db_user=daily, db_name=ty_analytic)\n",2019-09-01 11:48:49,2019-09-01 12:46:24,"{'date': '2019-09-01_09-29-01', 'prev_date': '2019-08-30_13-45-01', 'chunk': 'prep', 'sql_filename': 'review', 'kvs': '(('table_name', u'review_2yold'),)', 'target_filename': 'review_2yold', 'db_host': 'db.trustyou.com', 'db_port': '5432', 'db_user': 'daily', 'db_name': 'ty_analytic'}",prep,review,"(('table_name', u'review_2yold'),)",review_2yold
14602,daily.2019-09-01_09-29-01.csv,DumpTask,"DumpTask(date=2019-09-01_09-29-01, prev_date=2019-08-30_13-45-01, chunk=prep, sql_filename=review, kvs=(('table_name', u'review_2015_3'),), target_filename=review_2015_3, db_host=db.trustyou.com, db_port=5432, db_user=daily, db_name=ty_analytic)\n",2019-09-01 11:40:39,2019-09-01 12:03:41,"{'date': '2019-09-01_09-29-01', 'prev_date': '2019-08-30_13-45-01', 'chunk': 'prep', 'sql_filename': 'review', 'kvs': '(('table_name', u'review_2015_3'),)', 'target_filename': 'review_2015_3', 'db_host': 'db.trustyou.com', 'db_port': '5432', 'db_user': 'daily', 'db_name': 'ty_analytic'}",prep,review,"(('table_name', u'review_2015_3'),)",review_2015_3


In [322]:
585/8


73.125

In [323]:
clean_case_activity_selection.groupby(['sql_filename']).size().reset_index(name='counts').sort_values(by=['counts'])

,sql_filename,counts
1,daily_urls,13
2,mgmt_response,13
3,other_urls,13
4,regular_urls,13
5,reprocess_urls,13
7,source,13
0,consolidated_email,39
6,review,468


In [324]:
clean_case_activity_selection.groupby(['chunk']).size().reset_index(name='counts').sort_values(by=['counts'])

,chunk,counts
0,01,13
1,02,13
2,prep,559


In [325]:
clean_case_activity_selection.groupby(['target_filename']).size().reset_index(name='counts').sort_values(by=['counts'])

,target_filename,counts
0,review_2011_1,13
20,review_2016_1,13
21,review_2016_2,13
22,review_2016_3,13
23,review_2016_4,13
24,review_2017_1,13
25,review_2017_2,13
19,review_2015_4,13
26,review_2017_3,13
28,review_2018_1,13


In [327]:
clean_case_activity_selection.groupby(['kvs']).size().reset_index(name='counts').sort_values(by=['counts'])

,kvs,counts
0,"(('table_name', u'review_2011_1'),)",13
20,"(('table_name', u'review_2016_1'),)",13
21,"(('table_name', u'review_2016_2'),)",13
22,"(('table_name', u'review_2016_3'),)",13
23,"(('table_name', u'review_2016_4'),)",13
24,"(('table_name', u'review_2017_1'),)",13
25,"(('table_name', u'review_2017_2'),)",13
19,"(('table_name', u'review_2015_4'),)",13
26,"(('table_name', u'review_2017_3'),)",13
28,"(('table_name', u'review_2018_1'),)",13
